In [1]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/peft.git git+https://github.com/huggingface/transformers.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
import torch 
torch.cuda.is_available()

True

In [4]:
import os 
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch 
import torch.nn as nn
import bitsandbytes as bnb

from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(
    "bigscience/bloom-560m",
    # torch_dtype = torch.float16,
    device_map ='auto',
     load_in_8bit=True
  )

tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-560m")



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('8013'), PosixPath('//172.28.0.1')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

In [5]:
print(model)

BloomForCausalLM(
  (transformer): BloomModel(
    (word_embeddings): Embedding(250880, 1024)
    (word_embeddings_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0-23): 24 x BloomBlock(
        (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear8bitLt(in_features=1024, out_features=3072, bias=True)
          (dense): Linear8bitLt(in_features=1024, out_features=1024, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear8bitLt(in_features=1024, out_features=4096, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear8bitLt(in_features=4096, out_features=1024, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementw

In [6]:
for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

In [8]:
def print_trainable_params(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )
# show trainable params
print_trainable_params(model)

trainable params: 0 || all params: 559214592 || trainable%: 0.0


In [9]:
from peft import LoraConfig, get_peft_model 

config = LoraConfig( 
    r=8, #attention heads
    target_modules=["query_key_value"],            
    lora_alpha=16, #alpha scaling
    lora_dropout=0.05, #dropouts
    bias="none",
    task_type="CAUSAL_LM" # set this for CAUSAL LANGUAGE MODELS (like Bloom, LLaMA) or SEQ TO SEQ (like FLAN, T5)
)

model = get_peft_model(model, config)
print_trainable_params(model)

trainable params: 786432 || all params: 560001024 || trainable%: 0.14043402892063284


##**Dataset**

In [10]:
from datasets import load_dataset
qa_dataset = load_dataset("squad_v2")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

Dataset squad_v2 downloaded and prepared to /root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [13]:
def create_prompt(context, question, answer):
  if len(answer['text']) < 1 :
    answer = "can not find answer"

  else : 
    answer = answer["text"][0]

  prompt_template = f"### CONTEXT\n{context}\n\n### QUESTION\n{question}\n\n### ANSWER\n{answer}</s>"

  return prompt_template

mapped_qa_dataset = qa_dataset.map(lambda samples: tokenizer(create_prompt(samples['context'], samples['question'], samples['answers'])))


Map:   0%|          | 0/130319 [00:00<?, ? examples/s]

Map:   0%|          | 0/11873 [00:00<?, ? examples/s]

In [16]:
import transformers
trainer = transformers.Trainer(
    model=model, 
    train_dataset=mapped_qa_dataset['train'],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4, 
        gradient_accumulation_steps=4,
        warmup_steps=100, 
        max_steps=200, 
        learning_rate=2e-4, 
        fp16=True,
        logging_steps=1, 
        output_dir='outputs'
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

Step,Training Loss
1,2.732700
2,3.025000
3,2.805800
4,3.031500
5,2.914000
6,2.883800
7,2.875700
8,2.917700
9,2.958400
10,2.824500


Step,Training Loss
1,2.732700
2,3.025000
3,2.805800
4,3.031500
5,2.914000
6,2.883800
7,2.875700
8,2.917700
9,2.958400
10,2.824500


TrainOutput(global_step=200, training_loss=2.869536885023117, metrics={'train_runtime': 733.5055, 'train_samples_per_second': 4.363, 'train_steps_per_second': 0.273, 'total_flos': 1514289545183232.0, 'train_loss': 2.869536885023117, 'epoch': 0.02})

In [19]:
HUGGING_FACE_USERNAME = "Mohamedsheded33"

In [18]:
from huggingface_hub import notebook_login

notebook_login()

In [20]:
MODEL_NAME = "qa-bloom560-LoRA"
model.push_to_hub(f"{HUGGING_FACE_USERNAME}/{MODEL_NAME}",
                  use_auth_token=True
                  )

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.bin:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Mohamedsheded33/qa-bloom560-LoRA/commit/9842cd820ea52aef53083aa1c2f2025dca5c9a71', commit_message='Upload model', commit_description='', oid='9842cd820ea52aef53083aa1c2f2025dca5c9a71', pr_url=None, pr_revision=None, pr_num=None)

In [22]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = F"{HUGGING_FACE_USERNAME}/{MODEL_NAME}"
peft_config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(peft_config.base_model_name_or_path, return_dict=True, load_in_8bit=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(peft_config.base_model_name_or_path)

# Load the finetuned Lora PEFT model
qa_model = PeftModel.from_pretrained(model, peft_model_id)

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


In [ ]:
qa_model

In [39]:
from IPython.display import display, Markdown

def make_inference(context, question):
  batch = tokenizer(f"### CONTEXT\n{context}\n\n### QUESTION\n{question}\n\n### ANSWER\n", return_tensors ="pt")

  with torch.cuda.amp.autocast():
    output_tokens = qa_model.generate(**batch, max_new_tokens=200)\

  display(Markdown(tokenizer.decode(output_tokens[0],skip_special_tokens=True)))

In [47]:
context = "Al ahly beats zamalek as usual"
question = "Who is beaten by Al ahly?"
make_inference(context, question)

### CONTEXT
Al ahly beats zamalek as usual

### QUESTION
Who is beaten by Al ahly?

### ANSWER
zamalek

In [48]:
context = "Al ahly beats zamalek as usual"
question = "how far is the moon?"
make_inference(context, question)

### CONTEXT
Al ahly beats zamalek as usual

### QUESTION
how far is the moon?

### ANSWER
can not find answer

In [53]:
context = "1+1 equals 2"
question = "what is 1+1 equal ?"
make_inference(context, question)

### CONTEXT
1+1 equals 2

### QUESTION
what is 1+1 equal ?

### ANSWER
can not find answer